# Test 3 --- Final Test for 06.11.2025

In [1]:
# Install TensorBoard for visualization
%pip install tensorboard
%pip install tabulate

# Install required packages for reinforcement learning with Atari environments
%pip install ale-py
%pip install gymnasium[atari,accept-rom-license]
%pip install autorom
%pip install stable-baselines3

# Install imageio for video recording
%pip install imageio[ffmpeg]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Download Atari ROMs
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	C:\Users\PiePie\AppData\Local\Programs\Python\Python311\Lib\site-packages\AutoROM\roms

Existing ROMs will be overwritten.


# Install the Gym

In [3]:
import ale_py
import gymnasium as gym

# Configure the model save folder

In [4]:
import os
save_dir = "models/"
os.makedirs(save_dir, exist_ok=True)

# Now Model

In [5]:
from dataclasses import dataclass
import argparse
import time
from datetime import datetime
import numpy as np
import collections
import typing as tt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard.writer import SummaryWriter

import imageio
from IPython.display import HTML, display
from base64 import b64encode

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="imageio")
os.environ["IMAGEIO_FFMPEG_LOGLEVEL"] = "quiet"  # or "quiet"

In [95]:
#dqn_model
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

In [96]:
#wrappers

from gymnasium import spaces
from stable_baselines3.common import atari_wrappers


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_env(env_name: str, n_steps=4, render_mode=None, **kwargs):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode, **kwargs)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env

In [147]:
# Only rerun if it hasn't been run before; Prevents resetting the experiment log

has_run = 'config_history' in globals() and 'version_counter' in globals()

# Experiment logging utility
from IPython.display import Markdown, clear_output, display
import pandas as pd
import datetime

# Persistent experiment history
if not has_run:
    config_history = []
    version_counter = 0

# Log experiment configurations
def log_experiment(config_dict, note=""):
    """Log experiment configuration and display the table."""
    global version_counter, config_history

    """Append config to table and display nicely. Only if it's new."""
    if(config_history and all(config_dict.get(k) == config_history[-1].get(k) for k in config_dict)):
        return  # Skip logging if the config is the same as the last one

    entry = {
        "version": version_counter,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        **config_dict,
        "note": note
    }
    # Append the new entry to the history
    config_history.append(entry)
    version_counter += 1

    # Display the updated experiment log
    clear_output(wait=True)
    df = pd.DataFrame(config_history)
    display(Markdown(df.to_markdown(index=False)))


performance_history = []

def log_model_performance(frame_index, model_name, performance_metric_dict):
    global performance_history, performance_display

    entry = {
        "Frame Index": frame_index,
        "model_name": model_name,
        **performance_metric_dict,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    performance_history.append(entry)

    # Update table in-place WITHOUT clearing the whole cell
    df = pd.DataFrame(performance_history)
    markdown_table = df.to_markdown(index=False)
    performance_display.update(Markdown(markdown_table))

def remove_last_entry():
    """Remove the last entry from the experiment log."""
    global config_history, version_counter
    if config_history:
        config_history.pop()
        version_counter -= 1
        clear_output(wait=True)
        df = pd.DataFrame(config_history)
        display(Markdown(df.to_markdown(index=False)))

def clear_table():
    """Clear the experiment log table."""
    global config_history, version_counter
    config_history = []
    version_counter = 0
    clear_output(wait=True)

def clear_performance_table():
    """Clear the performance log table."""
    global performance_history
    performance_history = []
    clear_output(wait=True)

def print_table_for_github():
    """Print the experiment log table in markdown format for GitHub."""
    df = pd.DataFrame(config_history)
    print(df.to_markdown(index=False))

def snapshot_performance_table(run_label: str = ""):
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    label = f"_{run_label}" if run_label else ""
    os.makedirs("snapshots", exist_ok=True)

    perf_df = pd.DataFrame(performance_history)
    perf_path = f"snapshots/performance_history{label}_{timestamp}.md"
    perf_df.to_markdown(perf_path, index=False)

    print(f"Performance history snapshot saved to {perf_path}")

In [132]:
# Removes the last entry entered; only if I mean to
can_remove = True

if can_remove:
    remove_last_entry()

|   version | timestamp           | DEFAULT_ENV_NAME   |   MEAN_REWARD_BOUND |   GAMMA |   BATCH_SIZE |   REPLAY_SIZE |   LEARNING_RATE |   SYNC_TARGET_FRAMES |   REPLAY_START_SIZE |   SAVE_EPSILON |   EPSILON_DECAY_LAST_FRAME |   EPSILON_START |   EPSILON_FINAL | note                                                          |
|----------:|:--------------------|:-------------------|--------------------:|--------:|-------------:|--------------:|----------------:|---------------------:|--------------------:|---------------:|---------------------------:|----------------:|----------------:|:--------------------------------------------------------------|
|         0 | 2025-11-24 07:20:24 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run                                                  |
|         1 | 2025-11-24 08:46:48 | ALE/Pong-v5        |                  19 |    0.99 |           64 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Increase stability, try to help GPU                           |
|         2 | 2025-11-24 10:56:18 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0002 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Increased Learning rate to encourage more aggressive learning |
|         3 | 2025-11-24 15:07:35 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          7e-05  |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Decreased Learning rate to encourage more stabilized learning |
|         4 | 2025-11-24 21:33:35 | ALE/Pong-v5        |                  19 |    0.97 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Decreased gamma to prioritize more immediate rewards          |

In [99]:
# Clears the whole table, only if I mean to
can_clear = False

if can_clear:
    clear_table()

In [100]:
# Clears the whole table, only if I mean to
can_clear = False

if can_clear:
    clear_performance_table()

In [101]:
# Prints the table in markdown format
print_table_for_github()

In [157]:
# Base Configuration
config = {
    "DEFAULT_ENV_NAME": "ALE/Breakout-v5",
    "MEAN_REWARD_BOUND": 19,

    "GAMMA": 0.99,
    "BATCH_SIZE": 32,
    "REPLAY_SIZE": 10000,
    "LEARNING_RATE": 1e-4,
    "SYNC_TARGET_FRAMES": 1000,
    "REPLAY_START_SIZE": 10000,

    "SAVE_EPSILON": 0.5,  # Only save if at least this much better
    "EPSILON_DECAY_LAST_FRAME": 150000,
    "EPSILON_START": 1.0,
    "EPSILON_FINAL": 0.01
}

# Log automatically when you rerun this cell
log_experiment(config, note="Baseline run")

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]

|   version | timestamp           | DEFAULT_ENV_NAME    |   MEAN_REWARD_BOUND |   GAMMA |   BATCH_SIZE |   REPLAY_SIZE |   LEARNING_RATE |   SYNC_TARGET_FRAMES |   REPLAY_START_SIZE |   SAVE_EPSILON |   EPSILON_DECAY_LAST_FRAME |   EPSILON_START |   EPSILON_FINAL | note                                                                            |   Sync_TARGET_FRAMES |
|----------:|:--------------------|:--------------------|--------------------:|--------:|-------------:|--------------:|----------------:|---------------------:|--------------------:|---------------:|---------------------------:|----------------:|----------------:|:--------------------------------------------------------------------------------|---------------------:|
|         0 | 2025-11-24 07:20:24 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Baseline run                                                                    |                  nan |
|         1 | 2025-11-24 08:46:48 | ALE/Pong-v5         |                  19 |    0.99 |           64 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increase stability, try to help GPU                                             |                  nan |
|         2 | 2025-11-24 10:56:18 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0002 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increased Learning rate to encourage more aggressive learning                   |                  nan |
|         3 | 2025-11-24 15:07:35 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          7e-05  |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased Learning rate to encourage more stabilized learning                   |                  nan |
|         4 | 2025-11-24 21:33:35 | ALE/Pong-v5         |                  19 |    0.97 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased gamma to prioritize more immediate rewards                            |                  nan |
|         5 | 2025-11-24 22:59:08 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0001 |                  500 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased sync target frames to encourage more frequent updates                 |                  500 |
|         6 | 2025-11-25 05:17:38 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1500 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increased sync target frames to encourage less frequent and more stable updates |                  500 |
|         7 | 2025-11-25 09:21:56 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.005 | Decrease epsilon final to make agent greedier and less random                   |                  500 |
|         8 | 2025-11-25 10:46:36 | ALE/Pong-v5         |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.02  | Increase epsilon final to make agent less greedy and more random                |                  500 |
|         9 | 2025-11-25 13:27:01 | ALE/VideoPinball-v5 |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Baseline run                                                                    |                  nan |
|        10 | 2025-11-25 13:34:50 | ALE/VideoPinball-v5 |                5000 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Baseline run                                                                    |                  nan |
|        11 | 2025-11-25 13:49:41 | ALE/Breakout-v5     |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Baseline run                                                                    |                  nan |

In [ ]:
# ⚙️ Fast Training Config for Quick Test Run
#config["MEAN_REWARD_BOUND"] = 2
#config["REPLAY_START_SIZE"] = 1000
#config["EPSILON_DECAY_LAST_FRAME"] = 10000
#config["SYNC_TARGET_FRAMES"] = 500

config["BATCH_SIZE"] = 32               # 1 Increasing to 64 helped single dqn very much, harmed double dqn minorly
config["LEARNING_RATE"] = 1e-4          # 2 Increased to 2e-4 to have more aggressive learning, harmed both single and double dqn. Double did not learn
config["LEARNING_RATE"] = 1e-4          # 3 Decreased to 7e-5 to have more stablized learning, single-DQN seems to benefit and double-DQN is suffered majorly
config["GAMMA"] = 0.99                  # 4 Decreased gamma to 0.97 to prioritize more immediate rewards
config["SYNC_TARGET_FRAMES"] = 1000     # 5 Decreased to 500 to sync more frequently, minor benefit for single-DQN massive harm for double-DQN
config["SYNC_TARGET_FRAMES"] = 1000     # 6 Increased to 1500 to help stablize values, single-DQN benefited majorly while double-DQN suffer minor reduction
config["EPSILON_FINAL"] = 0.01         # 7 Decreased to 0.005 to make agent greedier, less random. single-DQN seemed to benefit while double-DQN suffered major reduction
config["EPSILON_FINAL"] = 0.01         # 8 Increased to 0.02 to make agent less greedy, more random.

# Log automatically when you rerun this cell
log_experiment(config, note="Increase epsilon final to make agent less greedy and more random")

|   version | timestamp           | DEFAULT_ENV_NAME   |   MEAN_REWARD_BOUND |   GAMMA |   BATCH_SIZE |   REPLAY_SIZE |   LEARNING_RATE |   SYNC_TARGET_FRAMES |   REPLAY_START_SIZE |   SAVE_EPSILON |   EPSILON_DECAY_LAST_FRAME |   EPSILON_START |   EPSILON_FINAL | note                                                                            |   Sync_TARGET_FRAMES |
|----------:|:--------------------|:-------------------|--------------------:|--------:|-------------:|--------------:|----------------:|---------------------:|--------------------:|---------------:|---------------------------:|----------------:|----------------:|:--------------------------------------------------------------------------------|---------------------:|
|         0 | 2025-11-24 07:20:24 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Baseline run                                                                    |                  nan |
|         1 | 2025-11-24 08:46:48 | ALE/Pong-v5        |                  19 |    0.99 |           64 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increase stability, try to help GPU                                             |                  nan |
|         2 | 2025-11-24 10:56:18 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0002 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increased Learning rate to encourage more aggressive learning                   |                  nan |
|         3 | 2025-11-24 15:07:35 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          7e-05  |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased Learning rate to encourage more stabilized learning                   |                  nan |
|         4 | 2025-11-24 21:33:35 | ALE/Pong-v5        |                  19 |    0.97 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased gamma to prioritize more immediate rewards                            |                  nan |
|         5 | 2025-11-24 22:59:08 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                  500 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Decreased sync target frames to encourage more frequent updates                 |                  500 |
|         6 | 2025-11-25 05:17:38 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1500 |               10000 |            0.5 |                     150000 |               1 |           0.01  | Increased sync target frames to encourage less frequent and more stable updates |                  500 |
|         7 | 2025-11-25 09:21:56 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.005 | Decrease epsilon final to make agent greedier and less random                   |                  500 |
|         8 | 2025-11-25 10:46:36 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |           0.02  | Increase epsilon final to make agent less greedy and more random                |                  500 |

In [149]:
# When an index value is passed in, make a change to the config

def next_change(index):
    # Reset all possible changes before next experiment
    config["BATCH_SIZE"] = 32
    config["LEARNING_RATE"] = 1e-4
    config["GAMMA"] = 0.99
    config["SYNC_TARGET_FRAMES"] = 1000
    config["EPSILON_FINAL"] = 0.01

    match index:
        case 1:
            config["BATCH_SIZE"] = 64               # 1 Increasing to 64 helped single dqn very much, harmed double dqn minorly
        case 2:
            config["LEARNING_RATE"] = 2e-4          # 2 Increased to 2e-4 to have more aggressive learning, harmed both single and double dqn. Double did not learn
        case 3:
            config["LEARNING_RATE"] = 7e-5          # 3 Decreased to 7e-5 to have more stablized learning, single-DQN seems to benefit and double-DQN is suffered majorly
        case 4:
            config["GAMMA"] = 0.97                  # 4 Decreased gamma to 0.97 to prioritize more immediate rewards
        case 5:
            config["SYNC_TARGET_FRAMES"] = 500     # 5 Decreased to 500 to sync more frequently, minor benefit for single-DQN massive harm for double-DQN
        case 6:
            config["SYNC_TARGET_FRAMES"] = 1500     # 6 Increased to 1500 to help stablize values, single-DQN benefited majorly while double-DQN suffer minor reduction
        case 7:
            config["EPSILON_FINAL"] = 0.005         # 7 Decreased to 0.005 to make agent greedier, less random. single-DQN seemed to benefit while double-DQN suffered major reduction
        case 8:
            config["EPSILON_FINAL"] = 0.02         # 8 Increased to 0.02 to make agent less greedy, more random.
        case _:
            print("Running baseline")

In [160]:
import os
from pathlib import Path

# Define directories
save_dir_local = "saved_models"

# Create both directories if they don't exist
os.makedirs(save_dir_local, exist_ok=True)

# Safe model filename
env_name = config["DEFAULT_ENV_NAME"]
safe_env_name = env_name.replace("/", "_")

In [105]:
@dataclass
class Experience:
    state: State
    action: Action
    reward: float
    done_trunc: bool
    new_state: State


class ExperienceBuffer:
    def __init__(self, capacity: int):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience: Experience):
        self.buffer.append(experience)

    def sample(self, batch_size: int) -> tt.List[Experience]:
        indices = np.random.choice(len(self), batch_size, replace=False)
        return [self.buffer[idx] for idx in indices]

In [106]:
class Agent:
    def __init__(self, env: gym.Env, exp_buffer: ExperienceBuffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state: tt.Optional[np.ndarray] = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net: DQN, device: torch.device, epsilon: float = 0.0) -> tt.Optional[float]:
        done_reward = None

        # epsilon-greedy action
        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            state_v = torch.as_tensor(self.state).unsqueeze(0).to(device)  # [1, C, H, W]
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # step env
        new_state, reward, done, trunc, _ = self.env.step(action)
        self.total_reward += reward

        # store raw numpy states in buffer
        self.exp_buffer.append(Experience(
            state=self.state,
            action=action,
            reward=float(reward),
            done_trunc=done or trunc,
            new_state=new_state,
        ))

        self.state = new_state

        if done or trunc:
            done_reward = self.total_reward
            self._reset()

        return done_reward

In [107]:
def batch_to_tensors(batch: tt.List[Experience], device: torch.device) -> BatchTensors:
    states, actions, rewards, dones, new_state = [], [], [], [], []
    for e in batch:
        states.append(e.state)
        actions.append(e.action)
        rewards.append(e.reward)
        dones.append(e.done_trunc)
        new_state.append(e.new_state)
    states_t = torch.as_tensor(np.asarray(states))
    actions_t = torch.LongTensor(actions)
    rewards_t = torch.FloatTensor(rewards)
    dones_t = torch.BoolTensor(dones)
    new_states_t = torch.as_tensor(np.asarray(new_state))
    return states_t.to(device), actions_t.to(device), rewards_t.to(device), \
           dones_t.to(device),  new_states_t.to(device)

In [108]:
USE_DOUBLE_DQN = False #global setting to toggle double dqn

def set_double_dqn(use_double: bool):
    global USE_DOUBLE_DQN
    USE_DOUBLE_DQN = use_double

In [109]:
def calc_loss(batch: tt.List[Experience], net: DQN, tgt_net: DQN,
              device: torch.device) -> torch.Tensor:
    states_t, actions_t, rewards_t, dones_t, new_states_t = batch_to_tensors(batch, device)

    state_action_values = net(states_t).gather(
        1, actions_t.unsqueeze(-1)
    ).squeeze(-1)

    # Compute the expected Q values
    with torch.no_grad():
        if USE_DOUBLE_DQN: # Double DQN
            next_actions = net(new_states_t).argmax(dim=1)
            next_state_values = tgt_net(new_states_t).gather(1, next_actions.unsqueeze(-1)).squeeze(-1)
        else: # DQN
            next_state_values = tgt_net(new_states_t).max(1)[0]

        next_state_values[dones_t] = 0.0

    expected_state_action_values = rewards_t + config["GAMMA"] * next_state_values; 

    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [110]:
def record_policy_video(env_name, model, device, video_filename="policy_rollout.mp4", max_steps=2000):
    """Run one greedy episode using the given model and save a video."""
    # Use your make_env to preserve preprocessing
    video_env = make_env(env_name, render_mode="rgb_array")
    state, _ = video_env.reset(seed=42)
    frames, total_reward = [], 0.0

    for step in range(max_steps):
        # Capture a rendered RGB frame (already handled by render_mode)
        frame = video_env.render()
        if isinstance(frame, np.ndarray):
            frames.append(frame)

        # Greedy action
        state_v = torch.as_tensor(state).unsqueeze(0).to(device)
        q_vals_v = model(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())

        state, reward, terminated, truncated, _ = video_env.step(action)
        total_reward += reward
        if terminated or truncated:
            frames.append(video_env.render())
            break

    video_env.close()

    # Save video
    if frames:
        os.makedirs(os.path.dirname(video_filename), exist_ok=True)

        # 🔹 Pad frames so height is divisible by 16 (prevents FFMPEG warnings)
        padded_frames = []
        for f in frames:
            h, w = f.shape[:2]
            new_h = (h + 15) // 16 * 16  # round up to nearest multiple of 16
            if new_h != h:
                pad = np.zeros((new_h - h, w, 3), dtype=f.dtype)
                f = np.vstack([f, pad])
            padded_frames.append(f)

        # 🔹 Write padded frames quietly
        with imageio.get_writer(video_filename, fps=30) as writer:
            for f in padded_frames:
                writer.append_data(f)

        print(f"Saved gameplay video: {video_filename} | Total reward: {total_reward:.2f}")
    else:
        print("No frames captured — render_mode may not be supported.")



In [111]:
import torch
print("torch.cuda.is_available():", torch.cuda.is_available())
print("torch.cuda.device_count():", torch.cuda.device_count())
print("torch.cuda.current_device():", torch.cuda.current_device() if torch.cuda.is_available() else None)
print("torch.cuda.get_device_name(0):", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)

torch.cuda.is_available(): True
torch.cuda.device_count(): 1
torch.cuda.current_device(): 0
torch.cuda.get_device_name(0): NVIDIA GeForce RTX 2080


In [161]:
# Main training loop with configuration changes
for i in range(0,9):
    clear_performance_table()
    next_change(i)

    # Run both DQN and Double DQN for each config change
    for j in range(0,2):
        if j == 0:
            set_double_dqn(False)
        else:
            set_double_dqn(True)

        # Training setup
        model_comment = (
            f"test_epsdec{config['EPSILON_DECAY_LAST_FRAME']}"
            f"_rs{config['REPLAY_START_SIZE']}"
            f"_sync{config['SYNC_TARGET_FRAMES']}"
        )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        # Environment and networks
        env = make_env(env_name)
        net = DQN(env.observation_space.shape, env.action_space.n).to(device)
        tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
        writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
        print(net)

        # Experience replay buffer and agent
        buffer = ExperienceBuffer(config["REPLAY_SIZE"])
        agent = Agent(env, buffer)
        epsilon = config["EPSILON_START"]

        # Optimizer setup
        optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
        total_rewards = []
        frame_idx = 0
        ts_frame = 0
        ts = time.time()
        best_m_reward = None

        if USE_DOUBLE_DQN:
            model_name = "Double DQN"
        else:
            model_name = "DQN"

        performance_metric = {
            "Finished Games": 0,
            "Reward": 0.0,
            "Epsilon": 0.0,
            "Speed (f/s)": 0.0,
            "Elapsed Time (min)": 0.0
        }

        # This creates a placeholder display area that we can update
        performance_display = display(Markdown(""), display_id=True)

        # Training loop
        start_time = time.time()
        last_fps_log = start_time
        last_fps_frame = frame_idx
        FPS_LOG_INTERVAL = 5.0  # seconds
        MAX_FRAMES = 300_000  # cap to keep run under ~1–2 hours
        while True:
            frame_idx += 1
            if frame_idx >= MAX_FRAMES:
                print(f"Stopping early at {frame_idx} frames (MAX_FRAMES reached).")

                # Save final video before exit
                timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
                video_filename = f"videos/Breakout/{model_name}/{safe_env_name}-final-{timestamp}.mp4"
                os.makedirs(f"videos/Breakout/{model_name}", exist_ok=True)
                record_policy_video(env_name, net, device, video_filename)
                break

            # Decay epsilon
            epsilon = max(config["EPSILON_FINAL"], config["EPSILON_START"] - frame_idx / config["EPSILON_DECAY_LAST_FRAME"])

            reward = agent.play_step(net, device, epsilon)
            if reward is not None:
                total_rewards.append(reward)
                speed = (frame_idx - ts_frame) / (time.time() - ts)
                elapsed = time.time() - start_time  # in seconds
                ts_frame = frame_idx
                ts = time.time()
                m_reward = np.mean(total_rewards[-100:])
                #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
                writer.add_scalar("epsilon", epsilon, frame_idx)
                writer.add_scalar("speed", speed, frame_idx)
                writer.add_scalar("reward_100", m_reward, frame_idx)
                writer.add_scalar("reward", reward, frame_idx)
                if best_m_reward is None or m_reward > best_m_reward + config["SAVE_EPSILON"]:
                    print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                        f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
                    
                    # Update performance metrics
                    performance_metric["Finished Games"] = len(total_rewards)
                    performance_metric["Reward"] = m_reward
                    performance_metric["Epsilon"] = epsilon
                    performance_metric["Speed (f/s)"] = speed
                    performance_metric["Elapsed Time (min)"] = elapsed / 60.0

                    # Log model performance
                    log_model_performance(frame_idx, model_name, performance_metric)

                    # Save the model
                    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
                    model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"
                    model_path_local = os.path.join(save_dir_local, model_filename)

                    torch.save(net.state_dict(), model_path_local)

                    print(f"Model saved to:\n - Local:        {model_path_local}")
                    if best_m_reward is not None:
                        print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
                    best_m_reward = m_reward

                    # Record first video of training
                    if len(total_rewards) == 1:
                        video_filename = f"videos/Breakout/{model_name}/{safe_env_name}-first-{int(m_reward)}-{timestamp}.mp4"
                        os.makedirs(f"videos/Breakout/{model_name}", exist_ok=True)
                        record_policy_video(env_name, net, device, video_filename)
                    
                if m_reward > config["MEAN_REWARD_BOUND"]:
                    print("Solved in %d frames!" % frame_idx)

                    # Final video recording before exit
                    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
                    video_filename = f"videos/Breakout/{model_name}/{safe_env_name}-solved-{int(m_reward)}-{timestamp}.mp4"
                    os.makedirs(f"videos/Breakout/{model_name}", exist_ok=True)
                    record_policy_video(env_name, net, device, video_filename)
                    break
            if len(buffer) < config["REPLAY_START_SIZE"]:
                continue
            if frame_idx % config["SYNC_TARGET_FRAMES"] == 0:
                tgt_net.load_state_dict(net.state_dict())

            optimizer.zero_grad()
            batch = buffer.sample(config["BATCH_SIZE"])
            loss_t = calc_loss(batch, net, tgt_net, device)
            loss_t.backward()
            optimizer.step()
        env.close()
        writer.close()

        # Since double dqn the last to be tested run, snapshot performance table
        if USE_DOUBLE_DQN:
            snapshot_performance_table(run_label=f"{model_name}_performance_{i}")

Using device: cuda
Creating environment ALE/Breakout-v5
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=4, bias=True)
  )
)


|   Frame Index | model_name   |   Finished Games |   Reward |   Epsilon |   Speed (f/s) |   Elapsed Time (min) | timestamp           |
|--------------:|:-------------|-----------------:|---------:|----------:|--------------:|---------------------:|:--------------------|
|             5 | DQN          |                1 |     0    | 0.999967  |       139.261 |          0.000581777 | 2025-11-26 02:36:24 |
|          6260 | DQN          |              770 |     0.51 | 0.958267  |       337.032 |          0.380234    | 2025-11-26 02:36:46 |
|         72428 | DQN          |             7311 |     1.03 | 0.517147  |       130.218 |          9.11635     | 2025-11-26 02:45:31 |
|         92170 | DQN          |             8600 |     1.56 | 0.385533  |       126.159 |         11.7798      | 2025-11-26 02:48:10 |
|        115415 | DQN          |             9852 |     2.07 | 0.230567  |       125.672 |         14.9161      | 2025-11-26 02:51:19 |
|        131404 | DQN          |            10554 |     2.59 | 0.123973  |       117.086 |         17.0836      | 2025-11-26 02:53:29 |
|        139510 | DQN          |            10859 |     3.11 | 0.0699333 |       130.166 |         18.1771      | 2025-11-26 02:54:34 |
|        156881 | DQN          |            11448 |     3.63 | 0.02      |       124.002 |         20.5306      | 2025-11-26 02:56:56 |
|        179951 | DQN          |            12174 |     4.14 | 0.02      |       122.719 |         23.655       | 2025-11-26 03:00:03 |

5: done 1 games, reward 0.000, eps 1.00, speed 139.26 f/s, time 0.0 min
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0236-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Breakout-v5
Saved gameplay video: videos/Breakout/DQN/ALE_Breakout-v5-first-0-20251126-0236.mp4 | Total reward: 0.00
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0236-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Breakout-v5
Saved gameplay video: videos/Breakout/DQN/ALE_Breakout-v5-first-0-20251126-0236.mp4 | Total reward: 0.00
6260: done 770 games, reward 0.510, eps 0.96, speed 337.03 f/s, time 0.4 min
6260: done 770 games, reward 0.510, eps 0.96, speed 337.03 f/s, time 0.4 min
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0236-test_epsdec150000_rs10000_sync1000.dat
Best reward updated 0.000 -> 0.510
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0236-test_e

|   Frame Index | model_name   |   Finished Games |   Reward |   Epsilon |   Speed (f/s) |   Elapsed Time (min) | timestamp           |
|--------------:|:-------------|-----------------:|---------:|----------:|--------------:|---------------------:|:--------------------|
|             5 | DQN          |                1 | 0        | 0.999967  |       139.261 |          0.000581777 | 2025-11-26 02:36:24 |
|          6260 | DQN          |              770 | 0.51     | 0.958267  |       337.032 |          0.380234    | 2025-11-26 02:36:46 |
|         72428 | DQN          |             7311 | 1.03     | 0.517147  |       130.218 |          9.11635     | 2025-11-26 02:45:31 |
|         92170 | DQN          |             8600 | 1.56     | 0.385533  |       126.159 |         11.7798      | 2025-11-26 02:48:10 |
|        115415 | DQN          |             9852 | 2.07     | 0.230567  |       125.672 |         14.9161      | 2025-11-26 02:51:19 |
|        131404 | DQN          |            10554 | 2.59     | 0.123973  |       117.086 |         17.0836      | 2025-11-26 02:53:29 |
|        139510 | DQN          |            10859 | 3.11     | 0.0699333 |       130.166 |         18.1771      | 2025-11-26 02:54:34 |
|        156881 | DQN          |            11448 | 3.63     | 0.02      |       124.002 |         20.5306      | 2025-11-26 02:56:56 |
|        179951 | DQN          |            12174 | 4.14     | 0.02      |       122.719 |         23.655       | 2025-11-26 03:00:03 |
|             5 | Double DQN   |                1 | 0        | 0.999967  |       208.898 |          0.000365686 | 2025-11-26 03:16:26 |
|           120 | Double DQN   |               11 | 0.636364 | 0.9992    |       340.619 |          0.00987518  | 2025-11-26 03:16:27 |
|         73507 | Double DQN   |             7264 | 1.14     | 0.509953  |       117.962 |          9.94019     | 2025-11-26 03:26:22 |
|         92757 | Double DQN   |             8506 | 1.65     | 0.38162   |       121.384 |         12.6894      | 2025-11-26 03:29:07 |
|        113884 | Double DQN   |             9676 | 2.21     | 0.240773  |       119.649 |         15.727       | 2025-11-26 03:32:10 |
|        129095 | Double DQN   |            10375 | 2.77     | 0.139367  |       116.44  |         17.9252      | 2025-11-26 03:34:21 |
|        143236 | Double DQN   |            10902 | 3.29     | 0.0450933 |       104.193 |         19.9682      | 2025-11-26 03:36:24 |
|        148268 | Double DQN   |            11056 | 3.82     | 0.02      |       117.097 |         20.6939      | 2025-11-26 03:37:08 |
|        185667 | Double DQN   |            12214 | 4.34     | 0.02      |       113.927 |         26.0984      | 2025-11-26 03:42:32 |
|        230309 | Double DQN   |            13606 | 4.88     | 0.02      |       112.292 |         32.5514      | 2025-11-26 03:48:59 |

5: done 1 games, reward 0.000, eps 1.00, speed 208.90 f/s, time 0.0 min
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0316-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Breakout-v5
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0316-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Breakout-v5
Saved gameplay video: videos/Breakout/Double DQN/ALE_Breakout-v5-first-0-20251126-0316.mp4 | Total reward: 0.00
Saved gameplay video: videos/Breakout/Double DQN/ALE_Breakout-v5-first-0-20251126-0316.mp4 | Total reward: 0.00
120: done 11 games, reward 0.636, eps 1.00, speed 340.62 f/s, time 0.0 min
120: done 11 games, reward 0.636, eps 1.00, speed 340.62 f/s, time 0.0 min
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0316-test_epsdec150000_rs10000_sync1000.dat
Best reward updated 0.000 -> 0.636
Model saved to:
 - Local:        saved_models\ALE_Breakout-v5-best_0-20251126-0

In [ ]:
#Clear output for next run
clear_performance_table()

# Training setup
model_comment = (
    f"test_epsdec{config['EPSILON_DECAY_LAST_FRAME']}"
    f"_rs{config['REPLAY_START_SIZE']}"
    f"_sync{config['SYNC_TARGET_FRAMES']}"
 )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Environment and networks
env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

# Experience replay buffer and agent
buffer = ExperienceBuffer(config["REPLAY_SIZE"])
agent = Agent(env, buffer)
epsilon = config["EPSILON_START"]

# Optimizer setup
optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

set_double_dqn(False)  # Set to True to use Double DQN

if USE_DOUBLE_DQN:
    model_name = "Double DQN"
else:
    model_name = "DQN"

performance_metric = {
    "Finished Games": 0,
    "Reward": 0.0,
    "Epsilon": 0.0,
    "Speed (f/s)": 0.0,
    "Elapsed Time (min)": 0.0
}

# This creates a placeholder display area that we can update
performance_display = display(Markdown(""), display_id=True)

# Training loop
start_time = time.time()
last_fps_log = start_time
last_fps_frame = frame_idx
FPS_LOG_INTERVAL = 5.0  # seconds
MAX_FRAMES = 300_000  # cap to keep run under ~1–2 hours
while True:
    frame_idx += 1
    if frame_idx >= MAX_FRAMES:
        print(f"Stopping early at {frame_idx} frames (MAX_FRAMES reached).")

        # Save final video before exit
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
        video_filename = f"videos/{model_name}/{safe_env_name}-final-{timestamp}.mp4"
        os.makedirs(f"videos/{model_name}", exist_ok=True)
        record_policy_video(env_name, net, device, video_filename)
        break

    # Decay epsilon
    epsilon = max(config["EPSILON_FINAL"], config["EPSILON_START"] - frame_idx / config["EPSILON_DECAY_LAST_FRAME"])

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + config["SAVE_EPSILON"]:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            
            # Update performance metrics
            performance_metric["Finished Games"] = len(total_rewards)
            performance_metric["Reward"] = m_reward
            performance_metric["Epsilon"] = epsilon
            performance_metric["Speed (f/s)"] = speed
            performance_metric["Elapsed Time (min)"] = elapsed / 60.0

            # Log model performance
            log_model_performance(frame_idx, model_name, performance_metric)

            # Save the model
            timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_local)

            print(f"Model saved to:\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward

            # Record first video of training
            if len(total_rewards) == 1:
                video_filename = f"videos/Pinball/{model_name}/{safe_env_name}-first-{int(m_reward)}-{timestamp}.mp4"
                os.makedirs(f"videos/Pinball/{model_name}", exist_ok=True)
                record_policy_video(env_name, net, device, video_filename)
            
        if m_reward > config["MEAN_REWARD_BOUND"]:
            print("Solved in %d frames!" % frame_idx)

            # Final video recording before exit
            timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
            video_filename = f"videos/Pinball/{model_name}/{safe_env_name}-solved-{int(m_reward)}-{timestamp}.mp4"
            os.makedirs(f"videos/Pinball/{model_name}", exist_ok=True)
            record_policy_video(env_name, net, device, video_filename)
            break
    if len(buffer) < config["REPLAY_START_SIZE"]:
        continue
    if frame_idx % config["SYNC_TARGET_FRAMES"] == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(config["BATCH_SIZE"])
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

Using device: cuda
Creating environment ALE/Pong-v5
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


|   Frame Index | model_name   |   Finished Games |   Reward |   Epsilon |   Speed (f/s) |   Elapsed Time (min) | timestamp           |
|--------------:|:-------------|-----------------:|---------:|----------:|--------------:|---------------------:|:--------------------|
|           189 | DQN          |                1 |   -21    | 0.99874   |       380.871 |           0.00820402 | 2025-11-25 10:47:03 |
|         61833 | DQN          |              265 |   -20.47 | 0.58778   |       137.333 |           6.96032    | 2025-11-25 10:54:00 |
|        100114 | DQN          |              392 |   -19.96 | 0.332573  |       129.474 |          11.7678     | 2025-11-25 10:58:49 |
|        116585 | DQN          |              434 |   -19.45 | 0.222767  |       129.398 |          13.8876     | 2025-11-25 11:00:56 |
|        128836 | DQN          |              459 |   -18.9  | 0.141093  |       128.055 |          15.4816     | 2025-11-25 11:02:32 |
|        137404 | DQN          |              473 |   -18.38 | 0.0839733 |       128.203 |          16.601      | 2025-11-25 11:03:39 |
|        147576 | DQN          |              487 |   -17.81 | 0.02      |       125.607 |          17.9413     | 2025-11-25 11:04:59 |
|        157230 | DQN          |              501 |   -17.29 | 0.02      |       125.57  |          19.2158     | 2025-11-25 11:06:16 |
|        169478 | DQN          |              518 |   -16.74 | 0.02      |       124.988 |          20.8374     | 2025-11-25 11:07:53 |
|        177483 | DQN          |              529 |   -16.2  | 0.02      |       126.296 |          21.8959     | 2025-11-25 11:08:56 |
|        183915 | DQN          |              537 |   -15.69 | 0.02      |       126.217 |          22.744      | 2025-11-25 11:09:47 |
|        192616 | DQN          |              548 |   -15.15 | 0.02      |       125.772 |          23.8968     | 2025-11-25 11:10:56 |
|        207896 | DQN          |              568 |   -14.63 | 0.02      |       127.781 |          25.9151     | 2025-11-25 11:12:58 |
|        226463 | DQN          |              593 |   -14.12 | 0.02      |       127.015 |          28.3706     | 2025-11-25 11:15:25 |
|        235464 | DQN          |              604 |   -13.59 | 0.02      |       126.134 |          29.5603     | 2025-11-25 11:16:36 |
|        248422 | DQN          |              620 |   -13.07 | 0.02      |       125.095 |          31.2771     | 2025-11-25 11:18:19 |
|        279055 | DQN          |              659 |   -12.56 | 0.02      |       126.333 |          35.3259     | 2025-11-25 11:22:22 |
|        294072 | DQN          |              677 |   -12.01 | 0.02      |       127.425 |          37.3085     | 2025-11-25 11:24:21 |

189: done 1 games, reward -21.000, eps 1.00, speed 380.87 f/s, time 0.0 min
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-21-20251125-1047-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Pong-v5
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-21-20251125-1047-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Pong-v5
Saved gameplay video: videos/DQN/ALE_Pong-v5-first--21-20251125-1047.mp4 | Total reward: -21.00
Saved gameplay video: videos/DQN/ALE_Pong-v5-first--21-20251125-1047.mp4 | Total reward: -21.00
61833: done 265 games, reward -20.470, eps 0.59, speed 137.33 f/s, time 7.0 min
61833: done 265 games, reward -20.470, eps 0.59, speed 137.33 f/s, time 7.0 min
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-20-20251125-1054-test_epsdec150000_rs10000_sync1000.dat
Best reward updated -21.000 -> -20.470
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-20-20251125-1054-test_epsdec150000_rs10000_

In [ ]:
# Training setup
model_comment = (
    f"test_epsdec{config['EPSILON_DECAY_LAST_FRAME']}"
    f"_rs{config['REPLAY_START_SIZE']}"
    f"_sync{config['SYNC_TARGET_FRAMES']}"
 )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Environment and networks
env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

# Experience replay buffer and agent
buffer = ExperienceBuffer(config["REPLAY_SIZE"])
agent = Agent(env, buffer)
epsilon = config["EPSILON_START"]

# Optimizer setup
optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

set_double_dqn(True)  # Set to True to use Double DQN

if USE_DOUBLE_DQN:
    model_name = "Double DQN"
else:
    model_name = "DQN"

performance_metric = {
    "Finished Games": 0,
    "Reward": 0.0,
    "Epsilon": 0.0,
    "Speed (f/s)": 0.0,
    "Elapsed Time (min)": 0.0
}

# This creates a placeholder display area that we can update
performance_display = display(Markdown(""), display_id=True)

# Training loop
start_time = time.time()
last_fps_log = start_time
last_fps_frame = frame_idx
FPS_LOG_INTERVAL = 5.0  # seconds
MAX_FRAMES = 300_000  # cap to keep run under ~1–2 hours
while True:
    frame_idx += 1
    if frame_idx >= MAX_FRAMES:
        print(f"Stopping early at {frame_idx} frames (MAX_FRAMES reached).")

        # Save final video before exit
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
        video_filename = f"videos/{model_name}/{safe_env_name}-final-{timestamp}.mp4"
        os.makedirs(f"videos/{model_name}", exist_ok=True)
        record_policy_video(env_name, net, device, video_filename)
        break

    epsilon = max(config["EPSILON_FINAL"], config["EPSILON_START"] - frame_idx / config["EPSILON_DECAY_LAST_FRAME"])

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + config["SAVE_EPSILON"]:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            
            # Update performance metrics
            performance_metric["Finished Games"] = len(total_rewards)
            performance_metric["Reward"] = m_reward
            performance_metric["Epsilon"] = epsilon
            performance_metric["Speed (f/s)"] = speed
            performance_metric["Elapsed Time (min)"] = elapsed / 60.0

            # Log model performance
            log_model_performance(frame_idx, model_name, performance_metric)

            # Save the model
            timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_local)

            print(f"Model saved to:\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward

            # Record first video of training
            if len(total_rewards) == 1:
                video_filename = f"videos/Pinball/{model_name}/{safe_env_name}-first-{int(m_reward)}-{timestamp}.mp4"
                os.makedirs(f"videos/Pinball/{model_name}", exist_ok=True)
                record_policy_video(env_name, net, device, video_filename)
            
        if m_reward > config["MEAN_REWARD_BOUND"]:
            print("Solved in %d frames!" % frame_idx)

            # Final video recording before exit
            timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
            video_filename = f"videos/Pinball/{model_name}/{safe_env_name}-solved-{int(m_reward)}-{timestamp}.mp4"
            os.makedirs(f"videos/Pinball/{model_name}", exist_ok=True)
            record_policy_video(env_name, net, device, video_filename)
            break
    if len(buffer) < config["REPLAY_START_SIZE"]:
        continue
    if frame_idx % config["SYNC_TARGET_FRAMES"] == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(config["BATCH_SIZE"])
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

Using device: cuda
Creating environment ALE/Pong-v5
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


|   Frame Index | model_name   |   Finished Games |   Reward |   Epsilon |   Speed (f/s) |   Elapsed Time (min) | timestamp           |
|--------------:|:-------------|-----------------:|---------:|----------:|--------------:|---------------------:|:--------------------|
|           189 | DQN          |                1 |   -21    | 0.99874   |       380.871 |           0.00820402 | 2025-11-25 10:47:03 |
|         61833 | DQN          |              265 |   -20.47 | 0.58778   |       137.333 |           6.96032    | 2025-11-25 10:54:00 |
|        100114 | DQN          |              392 |   -19.96 | 0.332573  |       129.474 |          11.7678     | 2025-11-25 10:58:49 |
|        116585 | DQN          |              434 |   -19.45 | 0.222767  |       129.398 |          13.8876     | 2025-11-25 11:00:56 |
|        128836 | DQN          |              459 |   -18.9  | 0.141093  |       128.055 |          15.4816     | 2025-11-25 11:02:32 |
|        137404 | DQN          |              473 |   -18.38 | 0.0839733 |       128.203 |          16.601      | 2025-11-25 11:03:39 |
|        147576 | DQN          |              487 |   -17.81 | 0.02      |       125.607 |          17.9413     | 2025-11-25 11:04:59 |
|        157230 | DQN          |              501 |   -17.29 | 0.02      |       125.57  |          19.2158     | 2025-11-25 11:06:16 |
|        169478 | DQN          |              518 |   -16.74 | 0.02      |       124.988 |          20.8374     | 2025-11-25 11:07:53 |
|        177483 | DQN          |              529 |   -16.2  | 0.02      |       126.296 |          21.8959     | 2025-11-25 11:08:56 |
|        183915 | DQN          |              537 |   -15.69 | 0.02      |       126.217 |          22.744      | 2025-11-25 11:09:47 |
|        192616 | DQN          |              548 |   -15.15 | 0.02      |       125.772 |          23.8968     | 2025-11-25 11:10:56 |
|        207896 | DQN          |              568 |   -14.63 | 0.02      |       127.781 |          25.9151     | 2025-11-25 11:12:58 |
|        226463 | DQN          |              593 |   -14.12 | 0.02      |       127.015 |          28.3706     | 2025-11-25 11:15:25 |
|        235464 | DQN          |              604 |   -13.59 | 0.02      |       126.134 |          29.5603     | 2025-11-25 11:16:36 |
|        248422 | DQN          |              620 |   -13.07 | 0.02      |       125.095 |          31.2771     | 2025-11-25 11:18:19 |
|        279055 | DQN          |              659 |   -12.56 | 0.02      |       126.333 |          35.3259     | 2025-11-25 11:22:22 |
|        294072 | DQN          |              677 |   -12.01 | 0.02      |       127.425 |          37.3085     | 2025-11-25 11:24:21 |
|           189 | Double DQN   |                1 |   -21    | 0.99874   |       383.613 |           0.00819348 | 2025-11-25 11:25:12 |
|         45187 | Double DQN   |              196 |   -20.49 | 0.698753  |       129.199 |           4.90006    | 2025-11-25 11:30:05 |
|         71955 | Double DQN   |              294 |   -19.98 | 0.5203    |       127.158 |           8.38083    | 2025-11-25 11:33:34 |
|         98318 | Double DQN   |              372 |   -19.46 | 0.344547  |       124.746 |          11.8896     | 2025-11-25 11:37:04 |
|        116711 | Double DQN   |              415 |   -18.92 | 0.221927  |       121.176 |          14.3967     | 2025-11-25 11:39:35 |
|        124110 | Double DQN   |              430 |   -18.39 | 0.1726    |       121.068 |          15.4164     | 2025-11-25 11:40:36 |
|        135643 | Double DQN   |              452 |   -17.82 | 0.0957133 |       118.802 |          17.0113     | 2025-11-25 11:42:12 |
|        142087 | Double DQN   |              463 |   -17.26 | 0.0527533 |       118.825 |          17.9132     | 2025-11-25 11:43:06 |
|        149166 | Double DQN   |              473 |   -16.68 | 0.02      |       117.338 |          18.9106     | 2025-11-25 11:44:06 |
|        155353 | Double DQN   |              482 |   -16.16 | 0.02      |       118.76  |          19.7842     | 2025-11-25 11:44:58 |
|        161526 | Double DQN   |              490 |   -15.57 | 0.02      |       118.03  |          20.654      | 2025-11-25 11:45:50 |
|        166609 | Double DQN   |              496 |   -15.03 | 0.02      |       116.042 |          21.3746     | 2025-11-25 11:46:34 |
|        172506 | Double DQN   |              503 |   -14.47 | 0.02      |       118.004 |          22.207      | 2025-11-25 11:47:23 |
|        180518 | Double DQN   |              514 |   -13.94 | 0.02      |       118.837 |          23.338      | 2025-11-25 11:48:31 |
|        188625 | Double DQN   |              524 |   -13.41 | 0.02      |       119.177 |          24.4808     | 2025-11-25 11:49:40 |
|        195443 | Double DQN   |              533 |   -12.9  | 0.02      |       118.871 |          25.4461     | 2025-11-25 11:50:38 |
|        200075 | Double DQN   |              538 |   -12.29 | 0.02      |       117.97  |          26.1009     | 2025-11-25 11:51:17 |
|        206233 | Double DQN   |              546 |   -11.78 | 0.02      |       118.768 |          26.9666     | 2025-11-25 11:52:09 |
|        212503 | Double DQN   |              554 |   -11.26 | 0.02      |       118.181 |          27.8531     | 2025-11-25 11:53:02 |
|        220764 | Double DQN   |              564 |   -10.66 | 0.02      |       119.442 |          29.0163     | 2025-11-25 11:54:12 |
|        229945 | Double DQN   |              575 |   -10.13 | 0.02      |       118.492 |          30.3081     | 2025-11-25 11:55:30 |
|        236537 | Double DQN   |              582 |    -9.49 | 0.02      |       119.073 |          31.2349     | 2025-11-25 11:56:25 |
|        241864 | Double DQN   |              588 |    -8.97 | 0.02      |       118.946 |          31.9852     | 2025-11-25 11:57:10 |
|        256378 | Double DQN   |              605 |    -8.43 | 0.02      |       117.505 |          34.0261     | 2025-11-25 11:59:13 |
|        260317 | Double DQN   |              609 |    -7.88 | 0.02      |       119.072 |          34.5801     | 2025-11-25 11:59:46 |
|        264911 | Double DQN   |              614 |    -7.35 | 0.02      |       116.479 |          35.2269     | 2025-11-25 12:00:25 |
|        278067 | Double DQN   |              630 |    -6.84 | 0.02      |       120.133 |          37.0746     | 2025-11-25 12:02:16 |
|        287188 | Double DQN   |              640 |    -6.32 | 0.02      |       119.929 |          38.3543     | 2025-11-25 12:03:32 |

189: done 1 games, reward -21.000, eps 1.00, speed 383.61 f/s, time 0.0 min
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-21-20251125-1125-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Pong-v5
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-21-20251125-1125-test_epsdec150000_rs10000_sync1000.dat
Creating environment ALE/Pong-v5
Saved gameplay video: videos/Double DQN/ALE_Pong-v5-first--21-20251125-1125.mp4 | Total reward: -21.00
Saved gameplay video: videos/Double DQN/ALE_Pong-v5-first--21-20251125-1125.mp4 | Total reward: -21.00
45187: done 196 games, reward -20.490, eps 0.70, speed 129.20 f/s, time 4.9 min
45187: done 196 games, reward -20.490, eps 0.70, speed 129.20 f/s, time 4.9 min
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-20-20251125-1130-test_epsdec150000_rs10000_sync1000.dat
Best reward updated -21.000 -> -20.490
Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-20-20251125-1130-test_epsdec1

In [192]:
import os, pathlib
print("CWD:", os.getcwd())
print("runs exists:", os.path.isdir("runs"))
if os.path.isdir("runs"):
    print("runs contents:", os.listdir("runs"))
    for name in os.listdir("runs"):
        p = os.path.join("runs", name)
        print(" -", name, "| isdir:", os.path.isdir(p))
        if os.path.isdir(p):
            # list one level deeper
            inner = os.listdir(p)
            print("   inner:", inner[:5])

CWD: d:\Github\CSCI-166-Final-Project
runs exists: True
runs contents: ['Breakout', 'Nov24_07-20-24_DESKTOP-JV4U154-ALE', 'Nov24_07-55-37_DESKTOP-JV4U154-ALE', 'Nov24_08-52-01_DESKTOP-JV4U154-ALE', 'Nov24_09-37-01_DESKTOP-JV4U154-ALE', 'Nov24_10-58-12_DESKTOP-JV4U154-ALE', 'Nov24_12-00-34_DESKTOP-JV4U154-ALE', 'Nov24_15-08-17_DESKTOP-JV4U154-ALE', 'Nov24_15-57-07_DESKTOP-JV4U154-ALE', 'Nov24_21-33-49_DESKTOP-JV4U154-ALE', 'Nov24_22-10-56_DESKTOP-JV4U154-ALE', 'Nov24_22-59-59_DESKTOP-JV4U154-ALE', 'Nov24_23-57-56_DESKTOP-JV4U154-ALE', 'Nov25_05-18-13_DESKTOP-JV4U154-ALE', 'Nov25_05-56-22_DESKTOP-JV4U154-ALE', 'Nov25_09-22-07_DESKTOP-JV4U154-ALE', 'Nov25_10-00-08_DESKTOP-JV4U154-ALE', 'Nov25_10-47-02_DESKTOP-JV4U154-ALE', 'Nov25_11-25-11_DESKTOP-JV4U154-ALE', 'Nov25_13-53-05_DESKTOP-JV4U154-ALE', 'Nov25_14-33-13_DESKTOP-JV4U154-ALE', 'Nov25_15-16-05_DESKTOP-JV4U154-ALE', 'Nov25_16-04-24_DESKTOP-JV4U154-ALE', 'Nov25_16-54-19_DESKTOP-JV4U154-ALE', 'Nov25_17-34-30_DESKTOP-JV4U154-ALE', 'Nov

In [195]:
from tensorboard.backend.event_processing import event_accumulator
from torch.utils.tensorboard import SummaryWriter
import os

# Configure env-specific normalization for rewards
ENV_INFO = {
    "Breakout": {"min": 0.0, "max": 400.0},
    "Pong": {"min": -21.0, "max": 21.0},
}

def norm_reward(env, r):
    info = ENV_INFO[env]
    return (r - info["min"]) / (info["max"] - info["min"])

def norm_01(x, x_min, x_max):
    if x_max == x_min:
        return 0.0
    return (x - x_min) / (x_max - x_min)

src_root = "runs"
dst_root = "runs_norm"  # new logs go here

os.makedirs(dst_root, exist_ok=True)

for run_name in os.listdir(src_root):
    run_path = os.path.join(src_root, run_name)
    if not os.path.isdir(run_path):
        continue

    # Simple heuristic: detect env from run_name
    env = "Pong" if ("Pong" in run_name or "pong" in run_name) else "Breakout"

    ea = event_accumulator.EventAccumulator(run_path)
    try:
        ea.Reload()
    except Exception as e:
        print("Failed to load", run_name, ":", e)
        continue

    print(run_name, ea.Tags())

    scalar_tags = ea.Tags().get("scalars", [])
    if not scalar_tags:
        continue

    writer = SummaryWriter(log_dir=os.path.join(dst_root, run_name))

    # ----- normalize reward_100 and reward if present -----
    if "reward_100" in scalar_tags:
        for ev in ea.Scalars("reward_100"):
            r_norm = norm_reward(env, ev.value)
            writer.add_scalar("reward_100_norm", r_norm, ev.step)

    if "reward" in scalar_tags:
        for ev in ea.Scalars("reward"):
            r_norm = norm_reward(env, ev.value)
            writer.add_scalar("reward_norm", r_norm, ev.step)

    # ----- normalize epsilon to [0,1] if present -----
    if "epsilon" in scalar_tags:
        eps_vals = ea.Scalars("epsilon")
        eps_min = min(ev.value for ev in eps_vals)
        eps_max = max(ev.value for ev in eps_vals)
        for ev in eps_vals:
            e_norm = norm_01(ev.value, eps_min, eps_max)
            writer.add_scalar("epsilon_norm", e_norm, ev.step)

    # ----- normalize speed per run if present -----
    if "speed" in scalar_tags:
        sp_vals = ea.Scalars("speed")
        sp_min = min(ev.value for ev in sp_vals)
        sp_max = max(ev.value for ev in sp_vals)
        for ev in sp_vals:
            s_norm = norm_01(ev.value, sp_min, sp_max)
            writer.add_scalar("speed_norm", s_norm, ev.step)

    writer.close()
    print("Processed", run_name)

print("Done. Now point TensorBoard at 'runs_norm'.")

Nov24_07-20-24_DESKTOP-JV4U154-ALE {'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
Nov24_07-55-37_DESKTOP-JV4U154-ALE {'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
Nov24_08-52-01_DESKTOP-JV4U154-ALE {'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
Nov24_09-37-01_DESKTOP-JV4U154-ALE {'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
Nov24_10-58-12_DESKTOP-JV4U154-ALE {'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}
Nov24_12-00-34_DESKTOP-JV4U154-ALE {'images': [], 'audi

In [13]:
%reload_ext tensorboard

from tensorboard import manager
manager.get_all()   # show stale instances

[TensorBoardInfo(version='2.13.0', start_time=1764166536, pid=13180, port=6006, path_prefix='', logdir='runs/Pong', db='', cache_key='eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJydW5zL1BvbmciXSwiY29uZmlndXJlX2t3YXJncyI6e30sIndvcmtpbmdfZGlyZWN0b3J5IjoiZDpcXEdpdGh1YlxcQ1NDSS0xNjYtRmluYWwtUHJvamVjdCJ9'),
 TensorBoardInfo(version='2.13.0', start_time=1764168877, pid=17416, port=6006, path_prefix='', logdir='runs_norm', db='', cache_key='eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJydW5zX25vcm0iLCItLXBvcnQiLCI2MDA2Il0sImNvbmZpZ3VyZV9rd2FyZ3MiOnt9LCJ3b3JraW5nX2RpcmVjdG9yeSI6ImQ6XFxHaXRodWJcXENTQ0ktMTY2LUZpbmFsLVByb2plY3QifQ=='),
 TensorBoardInfo(version='2.13.0', start_time=1764166976, pid=26936, port=6006, path_prefix='', logdir='runs', db='', cache_key='eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJydW5zIiwiLS1wb3J0IiwiNjAwNiJdLCJjb25maWd1cmVfa3dhcmdzIjp7fSwid29ya2luZ19kaXJlY3RvcnkiOiJkOlxcR2l0aHViXFxDU0NJLTE2Ni1GaW5hbC1Qcm9qZWN0In0='),
 TensorBoardInfo(version='2.13.0', start_time=1764166433, pid=27732, port=6006, pat

In [14]:
!taskkill /F /PID 29556


ERROR: The process "29556" not found.


In [16]:
%reload_ext tensorboard

In [17]:
%load_ext tensorboard
%tensorboard --logdir runs --port 6007

%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 19836), started 0:00:19 ago. (Use '!kill 19836' to kill it.)

In [ ]:
!taskkill /F /IM tensorboard.exe

ERROR: The process "tensorboard.exe" not found.


: 